# <span style="color: blue;">Генераторы</span>

### Что такое генератор?

Генератор — это функция, которая использует не только оператор `return`, но и оператор `yield`.

В результате выполнения оператора `yield` работа функции приостанавливается, а не прерывается, как при использовании оператора `return`.

Пример:

In [ ]:
def g():
    print("Started")
    x = 42
    yield x  # точка остановки в генераторе ("пауза")
    x += 1
    yield x
    print("Done")

In [ ]:
type(g)

In [ ]:
gen = g()  # вроде бы функция ничего не возвращает?
type(gen)  # но получаем специальный объект типа `generator`

In [ ]:
next(gen)  # результат `next` -- то, что написано у `yield`

In [ ]:
next(gen)

In [ ]:
next(gen)  # что будет здесь?

Подробнее: http://python.org/dev/peps/pep-0255

### Примеры генераторов: unique

С помощью `yield` можно писать итераторы более удобно

In [ ]:
def unique(iterable, seen=None):
    seen = set(seen or [])
    for item in iterable:
        if item not in seen:
            seen.add(item)
            yield item  # каждый новый элемент будет возвращаться генератором

xs = [1, 1, 2, 3]
unique(xs)

In [ ]:
list(unique(xs))

Генераторы -- это итераторы, поэтому есть реализация оператора **`in`** по умолчанию.

In [ ]:
1 in unique(xs)

Генератор можно исчерпать _(также как и итератор)_

In [ ]:
def g():
    for i in range(5):
        print(i)
        yield i

gen = g()

In [ ]:
2 in gen

In [ ]:
2 in gen

### Примеры генераторов: map

In [ ]:
def map(func, iterable, *rest):
    for args in zip(iterable, *rest):
        yield func(*args)

xs = range(5)
map(lambda x: x * x, xs)

In [ ]:
list(map(lambda x: x * x, xs))

In [ ]:
9 in map(lambda x: x * x, xs)

### Примеры генераторов: chain

In [ ]:
def chain(*iterables):
    for iterable in iterables:
        for item in iterable:
            yield item

xs = range(3)
ys = [42]
chain(xs, ys)

In [ ]:
list(chain(xs, ys))

In [ ]:
42 in chain(xs, ys)

### Примеры генераторов: count и enumerate

In [ ]:
def count(start=0):
    while True:
        yield start
        start += 1

next(count())

In [ ]:
counter = count()
next(counter)

In [ ]:
next(counter)

In [ ]:
list(count())  # так лучше не делать :)

In [ ]:
def enumerate(iterable, start=0):
    pass  # как?

list(enumerate("abc"))
# хотим получить: [(0, 'a'), (1, 'b'), (2, 'c')]

**Ответ:**

&nbsp;

&nbsp;

&nbsp;

&nbsp;

_(просто чтобы не проскролить к этому моменту слишком быстро)_

In [ ]:
def enumerate(iterable, start=0):
    return zip(count(), iterable)

list(enumerate("abc"))

### Переиспользование генераторов

Основное правило переиспользования генераторов: **не делайте этого**.

In [ ]:
def g():
    yield 42

gen = g()
list(gen)

In [ ]:
list(gen) # не тут-то было!

Если вы хотите переиспользовать генератор, подумайте ещё раз.

Если вы уверены, что без переиспользования не обойтись, воспользуйтесь функцией `tee` из модуля `itertools`.

### Коллекции и генераторы

Генераторы позволяют компактно реализовывать метод `__iter__` у коллекций.

Рассмотрим уже знакомый нам класс бинарного дерева:

In [ ]:
class BinaryTree:
    def __init__(self, value, left=(), right=()):  # почему tuple(), а не None ?
        self.value = value
        self.left, self.right = left, right
        
    def __iter__(self):  # inorder
        for node in self.left:
            yield node.value
        yield self.value
        for node in self.right:
            yield node.value
            
for b in BinaryTree(10):
    print(b)

Плюс генераторов в том, что они позволяют обойтись без лишних классов, например, `InOrderIterator`.

### Выражения-генераторы

Напоминание: в Python есть генераторы списков, множеств и словарей.

Выражения-генераторы работают аналогичным образом, но не порождают коллекцию в процессе работы:

In [ ]:
gen = (x ** 2 for x in range(10 ** 42) if x % 2 == 1)
gen

In [ ]:
next(gen)

In [ ]:
list(filter(lambda x: x % 2 == 1,
            (x ** 2 for x in range(10))))

Если выражение-генератор — единственный аргумент функции, то скобки можно опустить:

In [ ]:
sum(x ** 2 for x in range(10) if x % 2 == 1)

### Выражение yield

Оператор `yield` можно использовать как выражение:

In [ ]:
def g():
    res = yield  # точка входа 1
    print("Got {!r}".format(res))
    res = yield 42  # точка входа 2
    print("Got {!r}".format(res))

gen = g()
next(gen) # "промотаем" до первого yield
next(gen) # "промотаем" до второго yield

In [ ]:
next(gen) # выполним оставшуюся часть генератора

На первый взгляд выражение `yield` выглядит бесполезно, но первое впечатление обманчиво.

### Интерфейс генераторов: send

Метод **`send`** возобновляет выполнение генератора и “отправляет” свой аргумент в следующий `yield`.

In [ ]:
gen = g()
gen.send("foobar")

**Ошибка:** Отправлять что-то можно только в инициализированный генератор.

Чтобы инициализировать генератор нужно "отправить" ему `None` (чтобы промотать до первого `yield`). 

Функция `next` делает именно это:

In [ ]:
gen = g()
next(gen)

В общем случае метод **`send`** делает следующее:
* отправляет свой аргумент в генератор (и его значение записывается в переменную слева от **`yield`**)
* вызывает **`next`** у генератора

Результатом метода **`send`** является следующее значение генератора (или исключение `StopIteration`, если такого значения нет).<br/>
_(то есть возвращает то же, что и **`next`**)_

In [ ]:
gen = g()
gen.send(None)  # ≡ next(gen)
gen.send("foobar")

In [ ]:
gen.send("boo")

### Интерфейс генераторов: throw

Метод **`throw`**:
* поднимает переданное исключение в месте, где генератор приостановил исполнение <br/>
_(т.е. как будто мы в текущей строчке с `yield` написали `raise`)_
* и возвращает следующее значение генератора (если исключение обработано)<br/>
_(также как это делали `next` и `send`)_

In [ ]:
def g():
    try:
        yield 42
    except Exception as e:
        yield e

gen = g()
next(gen)  # инициализация генератора

In [ ]:
gen.throw(ValueError, "something is wrong")

In [ ]:
gen.throw(RuntimeError, "another error")  # что будет здесь?

Если генератор не обработал брошенное в него исключение, то выполнение генератора прекращается и исключение передаётся наверх по стеку вызовов.

In [ ]:
def g():
    try:
        yield 42
    except Exception as e:
        yield 'error :('
        yield e

gen = g()
next(gen)  # инициализация генератора

In [ ]:
gen.throw(ValueError)

In [ ]:
gen.throw(RuntimeError)

### Интерфейс генераторов: close

Метод `close` поднимает специальное исключение `GeneratorExit` в месте, где генератор приостановил исполнение:

Т.к. `GeneratorExit` наследник от `BaseException`, то он не уязвим для "`except Exception:`"

In [ ]:
def g():
    try:
        yield 42
        yield 43
        yield 44
    finally:
        print("Done")

In [ ]:
gen = g()
next(gen)

In [ ]:
gen.close()

Если всё хорошо, то метод `close` завершает работу генератора и ничего не возвращает.

Что может пойти не так? Генератор может обработать исключение `GeneratorExit` и поднять другое исключение.

In [ ]:
def g2():
    try:
        yield 42
        yield 43
    except GeneratorExit:
        raise RuntimeError("Don't close me!")
        
gen_2 = g2()
next(gen_2)

In [ ]:
gen_2.close()

Нельзя обрабатывать `GeneratorExit` (и тем самым игнорировать его):

In [ ]:
def g3():
    try:
        yield 42
    except GeneratorExit:
        print("Trying to ignore...")
        yield -1
        
gen_3 = g3()
next(gen_3)

In [ ]:
gen_3.close()

### Генераторы ∼ сопрограммы (aka coroutines)

**Сопрограмма** — это программа, которая может иметь больше одной точки входа, а также поддерживает остановку и продолжение с сохранением состояния.

Звучит как определение генератора наоборот:

In [ ]:
def grep(pattern):
    print("Looking for {!r}".format(pattern))
    while True:
        line = yield
        if pattern in line:
            print(line)

gen = grep("Gotcha!")
next(gen)

In [18]:
gen.send("This line doesn't have what we're looking for")

In [ ]:
gen.send("This one does. Gotcha!")

Подробнее: http://dabeaz.com/coroutines

### Удобная инициализация сопрограмм

Прежде, чем начать работать с сопрограммой, её нужно инициализировать с помощью вызова функции `next`.

Объявим декоратор `coroutine`, который скроет эту деталь реализации:

In [ ]:
import functools

def coroutine(g):
    @functools.wraps(g)
    def inner(*args, **kwargs):
        gen = g(*args, **kwargs)
        next(gen)
        return gen
    return inner

@coroutine
def grep(pattern):
    print("Looking for {!r}".format(pattern))
    while True:
        line = yield
        if pattern in line:
            print(line)

gen = grep("Gotcha!")
gen.send("This line doesn't have what we're looking for")
gen.send("This one does. Gotcha!")
gen.send("One more line for ya!")

### Генераторы ∼ легкие потоки (aka green threads)

**Но об этом не сейчас**

### Оператор yield from

Оператор **`yield from`** позволяет делегировать выполнение другому генератору:

In [ ]:
def chain(*iterables):
    for iterable in iterables:
        yield from iterable

Любые вызовы методов **`send`** и **`throw`** у родительского генератора будут переданы вложенному генератору без изменений.

`yield from` полезен когда нужно "разбить" генератор на кусочки. Он заменяет цикл с `yield` внутри.

Подробнее: http://python.org/dev/peps/pep-0380

### Оператор return и исключение StopIteration

Кроме оператора **`yield`** в теле генератора можно использовать оператор **`return`**.

На человеческом языке использование **`return`** означает:<br/>
«У меня больше нет элементов, извини, возьми лучше вот это.»

In [ ]:
def g():
    yield 42
    return ['something']  # держи!

gen = g()
next(gen)

In [ ]:
next(gen)

### `return `&nbsp;&nbsp;` ≠ `&nbsp;&nbsp;` raise StopIteration`


Несмотря на схожесть, использование оператора **`return`** в генераторе не эквивалентно поднятию исключения **`StopIteration`**.

Контрпример:

In [28]:
def g():
    try:
        yield 42
        raise StopIteration([])  # ≠ return []
    except Exception as e:
        pass

### Использование return в выражении yield from

Оператор **`yield from`**, как и оператор **`yield`**, можно использовать в качестве выражения.

При этом значением выражения **`yield from`** будет значение атрибута **`value`** у поднятого вложенным генератором исключения **`StopIteration`**:

In [ ]:
def f():
    yield 42
    return ['something']

def g():
    res = yield from f()
    print("Got {!r}".format(res))

gen = g()
next(gen)

In [ ]:
next(gen, 'finished')

### Менеджеры контекста и генераторы: мотивация

Протокол менеджеров контекста требует реализации двух методов: `__enter__` и `__exit_`.

Если мы хотим, чтобы у менеджера было какое-то состояние, то мы вынуждены также добавить метод `__init__`.

В итоге получаем:

In [ ]:
class cd:
    def __init__(self, path):
        self.path = path
        
    def __enter__(self):
        self.saved_cwd = os.getcwd()
        os.chdir(self.path)
        
    def __exit__(self, *exc_info):
        os.chdir(self.saved_cwd)

**Проблема:** Весьма длинновато выходит..

### Менеджеры контекста и генераторы: @contextmanager

Декоратор `contextmanage` из модуля `contextlib` принимает генератор специального вида и строит по нему менеджер контекста

In [35]:
from contextlib import contextmanager

@contextmanager
def cd(path): # __init__
    # __enter__:
    old_path = os.getcwd() 
    os.chdir(path)
    try:
        yield # --------- как бы разрез (здесь вклинивается `with`)
    finally:
        # __exit__:
        os.chdir(old_path) 

Генераторы позволяют сократить количество синтаксического шума при реализации менеджеров контекста.

`try`...`finally` нужен так как в теле `with` может возникнуть исключение, а `__exit__` надо обработать в любом случае

### Ещё один пример использования @contextmanager

Метод `__enter__`, построенный декоратором `contextmanager`, возвращает аргумент оператора `yield`:

In [ ]:
import tempfile
import shutil

@contextmanager
def tempdir(): # __init__
    outdir = tempfile.mkdtemp() # __enter__
    try:
        yield outdir            # ---------
    finally:
        shutil.rmtree(outdir)   # __exit__
        
with tempdir() as path:
    print(path)

### Генераторы: резюме

Генератор в `Python` — это функция, которая использует операторы `yield` или `yield from`.

В мире `Python` генераторы вездесущи не менее, чем любимые всеми декораторы.

Мы поговорили о том, что генераторы можно использовать
* как итераторы
* как сопрограммы
* как легкие потоки
* для компактной реализации менеджеров контекста